Stats on Ca2+ imaging with miniscope and Vigilance States

In [40]:
import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib widget


In [41]:
# Specify the directory containing the Excel files
directory = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/"

# Initialize an empty list to store the dataframes
dfs = []
df=[]

NrSubtype='All' # to changed 

if NrSubtype=='L1':
    nametofind='VigilanceState_GlobalResults'
else:
    nametofind='VigilanceState_AllNr_GlobalResults'#for Purple, ThreeColDots and ThreeBlueLines

# Recursively traverse the directory structure
for root, _, files in os.walk(directory):
    for filename in files:
        # Check if the file is an Excel file and contains the specified name
        if filename.endswith('.xlsx') and nametofind in filename: 
            # Construct the full path to the file
            filepath = os.path.join(root, filename)
            # Read the Excel file into a dataframe and append it to the list
            df = pd.read_excel(filepath)
            dfs.append(df)

# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(dfs, ignore_index=True)

In [42]:
combined_df

,Unnamed: 0,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,Substate,SubstateNumber,DurationSubstate,CalciumActivity,AUC_calcium,Avg_CalciumActivity,Avg_AUC_calcium,SpikeActivity,AUC_spike,Avg_SpikeActivity,Avg_AUC_spike
0,0,Purple,session11,NaN,4,0,0,NREM,0,1017,2.246678,2281.178575,0.982022,20539.835565,0.010364,10.517748,0.003530,73.834710
1,1,Purple,session11,NaN,4,0,0,Wake,1,100,2.831403,277.537833,0.982022,20539.835565,0.013720,1.344034,0.003530,73.834710
2,2,Purple,session11,NaN,4,0,0,NREM,2,400,2.484344,988.561705,0.982022,20539.835565,0.007826,3.122411,0.003530,73.834710
3,3,Purple,session11,NaN,4,0,0,Wake,3,100,2.872802,281.685946,0.982022,20539.835565,0.015720,1.556321,0.003530,73.834710
4,4,Purple,session11,NaN,4,0,0,NREM,4,300,2.739895,816.298818,0.982022,20539.835565,0.009580,2.856783,0.003530,73.834710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18091,12078,ThreeColDotsOK,session34,NaN,332,7,86,NREM,36,1800,0.320192,576.025735,0.352124,8774.569221,0.003462,6.228945,0.003808,94.886472
18092,12079,ThreeColDotsOK,session34,NaN,332,7,86,N2,37,1050,0.679538,711.799036,0.352124,8774.569221,0.009302,9.758251,0.003808,94.886472
18093,12080,ThreeColDotsOK,session34,NaN,332,7,86,NREM,38,900,0.333541,298.163895,0.352124,8774.569221,0.002836,2.549932,0.003808,94.886472
18094,12081,ThreeColDotsOK,session34,NaN,332,7,86,REM,39,750,0.910496,680.825146,0.352124,8774.569221,0.009339,6.994671,0.003808,94.886472


In [43]:
#Remove non defined Unique Units 
combined_df = combined_df[combined_df['Unique_Unit'] != '[]']
combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(str)
combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
combined_df['UnitValue'] = combined_df['UnitValue'].astype(str)

C:\Users\Manip2\AppData\Local\Temp\ipykernel_35524\1303002081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(str)
C:\Users\Manip2\AppData\Local\Temp\ipykernel_35524\1303002081.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
C:\Users\Manip2\AppData\Local\Temp\ipykernel_35524\1303002081.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

Save big dataset for stats

In [44]:
filenameOut = f'{directory}{NrSubtype}_VigilanceStates_GrandAverageAB.xlsx'
writer = pd.ExcelWriter(filenameOut)
combined_df.to_excel(writer)
writer.close()

Do Stats

From Formation Python CRNL

In [ ]:
"""md = smf.mixedlm('AUC_calcium ~ Substate', combined_df, groups=combined_df['Unique_Unit']) 
## mixed lm: idem ols (avec formula) mais cette fois on rajoute une variable groups, qui indique le facteur de fixed effect
mdf = md.fit()
print(mdf.summary())
"""

From Chat GPT

In [ ]:
"""# Fit the mixed-effects model
mixedlm_model = sm.MixedLM.from_formula('AUC_spike ~ Substate', groups='Unique_Unit', data=combined_df)
result = mixedlm_model.fit()

# Print model summary
print(result.summary())"""

In [ ]:
"""# Get the coefficients from the model results
coefficients = result.params

# Plot the coefficients
plt.bar(coefficients.index, coefficients)
plt.xlabel('Coefficient')
plt.ylabel('Value')
plt.title('Coefficients of Mixed Effects Model')
plt.show()"""

From forum

In [ ]:
"""model = smf.mixedlm(formula='AUC_spike ~ Substate',
#                re_formula='~ Unique_Unit',
                data=combined_df, groups='Unique_Unit').fit()
print(model.summary())"""